In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import anndata as ad
import numpy as np
import os
import scanpy as sc
import numpy as np

from evaluate import evaluate
from prediction import ADTPredictor, ADTPredictorKRREnsemble

### Train on NeurIPS competition dataset

In [2]:
dataset_path = "../../../../../PycharmProjects/ModalityPrediction/datasets/openproblems_bmmc_cite_phase2_rna/openproblems_bmmc_cite_phase2_rna.censor_dataset.output_"
gex_train = ad.read_h5ad(dataset_path + "train_mod1.h5ad")
gex_test = ad.read_h5ad(dataset_path + "test_mod1.h5ad")
adt_train = ad.read_h5ad(dataset_path + "train_mod2.h5ad")
adt_test = ad.read_h5ad(dataset_path + "test_mod2.h5ad")

In [4]:
# using high-level interface
pipe = ADTPredictor(do_log1p=False)
# fit on training data
# gex_test is optional and is used for transductive preprocessing if provided
# gex_names and adt_names are optional and should refer to the variable names of gex_train and adt_train
# if not provided, the predict() method will assume that all the columns of the test GEX matrix are in the same order as the training GEX matrix
pipe.fit(gex_train=gex_train.X.toarray(), adt_train=adt_train.X.toarray(), gex_test=gex_test.X.toarray(), gex_names=gex_train.var_names, adt_names=adt_train.var_names)

In [7]:
adt_pred, adt_names = pipe.predict(gex_test.X.toarray())
evaluate(adt_pred, adt_test.X.toarray())

RMSE: 0.3834063
Pearson correlation: 0.873978837329393
Spearman correlation: 0.8403242514005046


(0.3834063, 0.873978837329393, 0.8403242514005046)

In [8]:
# adt names are also stored as a property
pipe.adt_names

Index(['CD86', 'CD274', 'CD270', 'CD155', 'CD112', 'CD47', 'CD48', 'CD40',
       'CD154', 'CD52',
       ...
       'CD94', 'CD162', 'CD85j', 'CD23', 'CD328', 'HLA-E', 'CD82', 'CD101',
       'CD88', 'CD224'],
      dtype='object', length=134)

In [10]:
# save the trained pipeline to a file
pipe.save("ADTPredictor_neuripstrain_alltypes.joblib")

### Train on NeurIPS competition dataset, only T cells

In [ ]:
dataset_with_celltype = "../../../../../PycharmProjects/ModalityPrediction/datasets/post_competition/openproblems_bmmc_cite_complete.h5ad"
gex_adt_with_celltype = ad.read_h5ad(dataset_with_celltype)

In [29]:
gex_adt_with_celltype.obs["cell_type"].value_counts()

CD14+ Mono                          21693
CD4+ T activated                     6966
CD4+ T naive                         5897
NK                                   5434
Reticulocyte                         4272
Erythroblast                         4039
Naive CD20+ B IGKC+                  3990
CD8+ T naive                         3107
CD16+ Mono                           2635
NK CD158e1+                          2167
Naive CD20+ B IGKC-                  1979
G/M prog                             1881
pDC                                  1758
HSC                                  1703
cDC2                                 1702
Lymph prog                           1681
Transitional B                       1575
Proerythroblast                      1512
CD8+ T CD57+ CD45RO+                 1470
Normoblast                           1435
CD8+ T CD57+ CD45RA+                 1303
CD8+ T TIGIT+ CD45RO+                1160
CD4+ T activated integrinB7+         1056
CD8+ T TIGIT+ CD45RA+             

In [3]:
def filter_to_Tcells(adata):
    gex_data = adata[:, adata.var["feature_types"] == "GEX"]
    adt_data = adata[:, adata.var["feature_types"] == "ADT"]
    cell_mask = adata.obs["cell_type"].str.contains("T ") | adata.obs["cell_type"].str.endswith("T")
    return gex_data[cell_mask], adt_data[cell_mask]

In [6]:
gex_train_Tcells, adt_train_Tcells = filter_to_Tcells(gex_adt_with_celltype[gex_train.obs_names])
gex_test_Tcells, adt_test_Tcells = filter_to_Tcells(gex_adt_with_celltype[gex_test.obs_names])

In [23]:
gex_train_Tcells.obs["cell_type"].value_counts()

CD4+ T activated                    4989
CD4+ T naive                        4294
CD8+ T naive                        2081
CD8+ T TIGIT+ CD45RO+                869
CD8+ T CD57+ CD45RA+                 853
CD8+ T TIGIT+ CD45RA+                781
CD4+ T activated integrinB7+         725
CD8+ T CD49f+                        594
CD8+ T CD69+ CD45RO+                 498
CD8+ T CD69+ CD45RA+                 456
MAIT                                 438
T reg                                393
CD8+ T CD57+ CD45RO+                 380
gdT CD158b+                          210
gdT TCRVD2+                          155
CD4+ T CD314+ CD45RA+                 83
dnT                                   49
CD8+ T naive CD127+ CD26- CD101-      37
T prog cycling                        18
Name: cell_type, dtype: int64

In [7]:
# first evaluate the performance on T cells using the model trained on all cell types
pipe = ADTPredictor(do_log1p=False)
pipe.load("ADTPredictor_neuripstrain_alltypes.joblib")
adt_pred, adt_names = pipe.predict(np.log1p(gex_test_Tcells.X.toarray()))
evaluate(adt_pred, adt_test_Tcells.X.toarray())

RMSE: 0.38097182
Pearson correlation: 0.86761132274652
Spearman correlation: 0.8393886096299725


(0.38097182, 0.86761132274652, 0.8393886096299725)

In [8]:
# train a new pipeline on only T cells
pipe_Tcells = ADTPredictor(do_log1p=True)
pipe_Tcells.fit(gex_train=gex_train_Tcells.X.toarray(),
                adt_train=adt_train_Tcells.X.toarray(),
                gex_test=gex_test_Tcells.X.toarray(),
                gex_names=gex_train_Tcells.var_names,
                adt_names=adt_train_Tcells.var_names)

In [9]:
# evaluate on T cells
adt_pred, adt_names = pipe_Tcells.predict(gex_test_Tcells.X.toarray())
evaluate(adt_pred, adt_test_Tcells.X.toarray())

RMSE: 0.37337393
Pearson correlation: 0.873068129111393
Spearman correlation: 0.850552356692189


(0.37337393, 0.873068129111393, 0.850552356692189)

In [10]:
# save the trained pipeline
pipe_Tcells.save("ADTPredictor_neuripstrain_Tcells.joblib")

In [ ]:
# load the blood tcells dataset, to test the generalization of the model
datasets_path = "../../../../../PycharmProjects/ModalityPrediction/datasets/"
gex_path = os.path.join(datasets_path, "bloodTcellsCITEseqDOGMAseq/GSM6032900_CITE_RNA_filtered_feature_bc_matrix.h5")
adt_path = os.path.join(datasets_path, "bloodTcellsCITEseqDOGMAseq/GSM6032898_CITE_ADT.csv.gz")

print("Loading gex, adt")
gex_tcellsdset = sc.read_10x_h5(gex_path)
adt_tcellsdset = ad.read_csv(adt_path, first_column_names=True).transpose()

print("Filtering duplicate variables, zero variables, zero cells, fixing adt var names")
# filter out duplicate variables
gex_tcellsdset.var_names_make_unique()
adt_tcellsdset.var_names_make_unique()
# filter out cells with no adt measurements
gex_tcellsdset = gex_tcellsdset[adt_tcellsdset.obs_names]
gex_tcellsdset_df = gex_tcellsdset.to_df()
# filter out genes with constant expression in every cell
gex_tcellsdset = gex_tcellsdset[:, (gex_tcellsdset_df != gex_tcellsdset_df.iloc[0]).any(axis=0)]
# delete the last 6 characters of the protein names (to match the protein names in the competition dataset)
adt_tcellsdset.var_names = [x[:-6] for x in adt_tcellsdset.var_names]

In [ ]:
# predict using the model trained on all cell types
# by providing gex_names, the predict() method will filter out the GEX variables on which the model was not trained
# the GEX variables that were in the training set but not in gex_names will be set to 0
adt_tcellsdset_pred, adt_tcellsdset_pred_names = pipe.predict(np.log1p(gex_tcellsdset.X.toarray()), gex_names=gex_tcellsdset.var_names)

In [14]:
# filter out the proteins that are not in the competition dataset
adt_tcellsdset_true = adt_tcellsdset[:, adt_tcellsdset_pred_names[np.isin(adt_tcellsdset_pred_names, adt_tcellsdset.var_names)]]

In [17]:
evaluate(adt_tcellsdset_pred[:, np.isin(adt_tcellsdset_pred_names, adt_tcellsdset_true.var_names)], adt_tcellsdset_true.X)

RMSE: 48.758392
Pearson correlation: 0.4744033198688313
Spearman correlation: 0.6558111443884448


(48.758392, 0.4744033198688313, 0.6558111443884448)

In [18]:
# predict using the model trained on T cells
adt_tcellsdset_pred, adt_tcellsdset_pred_names = pipe_Tcells.predict(gex_tcellsdset.X.toarray(), gex_names=gex_tcellsdset.var_names)
evaluate(adt_tcellsdset_pred[:, np.isin(adt_tcellsdset_pred_names, adt_tcellsdset_true.var_names)], adt_tcellsdset_true.X)

/home/ico/Documents/GitHub/scLinearDev/inst/python/prediction.py:139: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  gex_test = ad.AnnData(gex_test)


RMSE: 48.75221
Pearson correlation: 0.49283479922093953
Spearman correlation: 0.6872483489369596


(48.75221, 0.49283479922093953, 0.6872483489369596)

### Train on NeurIPS competition dataset, only B cells

In [20]:
def filter_to_Bcells(adata):
    gex_data = adata[:, adata.var["feature_types"] == "GEX"]
    adt_data = adata[:, adata.var["feature_types"] == "ADT"]
    cell_mask = adata.obs["cell_type"].str.contains("B ") | adata.obs["cell_type"].str.endswith("B")
    return gex_data[cell_mask], adt_data[cell_mask]

gex_train_Bcells, adt_train_Bcells = filter_to_Bcells(gex_adt_with_celltype[gex_train.obs_names])
gex_test_Bcells, adt_test_Bcells = filter_to_Bcells(gex_adt_with_celltype[gex_test.obs_names])

In [22]:
gex_train_Bcells.obs["cell_type"].value_counts()

Naive CD20+ B IGKC+    2464
Naive CD20+ B IGKC-    1205
Transitional B          922
B1 B IGKC+              541
B1 B IGKC-              407
Name: cell_type, dtype: int64

In [24]:
# train a new pipeline on only B cells
pipe_Bcells = ADTPredictor(do_log1p=True)
pipe_Bcells.fit(gex_train=gex_train_Bcells.X.toarray(),
                adt_train=adt_train_Bcells.X.toarray(),
                gex_test=gex_test_Bcells.X.toarray(),
                gex_names=gex_train_Bcells.var_names,
                adt_names=adt_train_Bcells.var_names)

In [25]:
# evaluate on B cells
adt_pred, adt_names = pipe_Bcells.predict(gex_test_Bcells.X.toarray())
evaluate(adt_pred, adt_test_Bcells.X.toarray())

RMSE: 0.33909285
Pearson correlation: 0.8973023063828811
Spearman correlation: 0.8576481839930153


(0.33909285, 0.8973023063828811, 0.8576481839930153)

In [26]:
# save the trained pipeline
pipe_Bcells.save("ADTPredictor_neuripstrain_Bcells.joblib")

### Train on NeurIPS competition dataset, only NK cells

In [30]:
def filter_to_NKcells(adata):
    gex_data = adata[:, adata.var["feature_types"] == "GEX"]
    adt_data = adata[:, adata.var["feature_types"] == "ADT"]
    cell_mask = adata.obs["cell_type"].str.contains("NK ") | adata.obs["cell_type"].str.endswith("NK")
    return gex_data[cell_mask], adt_data[cell_mask]

gex_train_NKcells, adt_train_NKcells = filter_to_NKcells(gex_adt_with_celltype[gex_train.obs_names])
gex_test_NKcells, adt_test_NKcells = filter_to_NKcells(gex_adt_with_celltype[gex_test.obs_names])

In [31]:
gex_train_NKcells.obs["cell_type"].value_counts()

NK             3657
NK CD158e1+    1683
Name: cell_type, dtype: int64

In [32]:
# train a new pipeline on only NK cells
pipe_NKcells = ADTPredictor(do_log1p=True)
pipe_NKcells.fit(gex_train=gex_train_NKcells.X.toarray(),
                adt_train=adt_train_NKcells.X.toarray(),
                gex_test=gex_test_NKcells.X.toarray(),
                gex_names=gex_train_NKcells.var_names,
                adt_names=adt_train_NKcells.var_names)

In [33]:
# evaluate on NK cells
adt_pred, adt_names = pipe_NKcells.predict(gex_test_NKcells.X.toarray())
evaluate(adt_pred, adt_test_NKcells.X.toarray())

RMSE: 0.34588972
Pearson correlation: 0.8842237778611798
Spearman correlation: 0.8582723676507771


(0.34588972, 0.8842237778611798, 0.8582723676507771)

In [34]:
# save the trained pipeline
pipe_NKcells.save("ADTPredictor_neuripstrain_NKcells.joblib")

### Load the pretrained kernel ridge regression ensemble model

In [3]:
pipe = ADTPredictorKRREnsemble(do_log1p=False)
pipe.load('ADTPredictorKRREnsemble_neuripstrain_alltypes.joblib')
adt_pred, adt_names = pipe.predict(gex_test.X.toarray())
evaluate(adt_pred, adt_test.X.toarray())

RMSE: 0.3555361
Pearson correlation: 0.8904932287572471
Spearman correlation: 0.8597986120552739


(0.3555361, 0.8904932287572471, 0.8597986120552739)